# DATASET HORACIO

### INSTALL LIBRARIES

In [1]:
import pandas as pd
import psycopg2 as pg

### POSTGRES CONNECTION

In [2]:
conn = pg.connect("dbname=postgres user=postgres password=saude123")

### IMPORT DATA

In [3]:
city_state = pd.read_sql_query ('''
select code6 as city_ibge_code_06,code7 as city_ibge_code_07,state from socioeconomic_cities
''', conn)

code7_6 = pd.read_sql_query ('''
select code7 as city_ibge_code_07,code6 as city_ibge_code_06 from socioeconomic_cities sc 
''', conn)

brasil_io = pd.read_sql_query ('''
select state,city_name,city_ibge_code_07,date,last_available_deaths as deaths,estimated_population from brasil_io 
where place_type = 'city'
order by city_name,date
''', conn)

str_index = pd.read_sql_query ('''
select 
state,
date, 
stringency_index 
from oxford_measures
where jurisdiction = 'STATE_GOV' and city_name is null or city_name = ''
order by state,date
''',conn)

vacc = pd.read_sql_query('''
select date,state,city_ibge_code_06,sum(doses) as doses from si_pni_reduced
where (vaccine_dose = 'D2' or vaccine_dose = 'D_unica')
group by date,state,city_ibge_code_06
''',conn)

### IBGE CODE

In [4]:
city_state['city_ibge_code_06'] = city_state['city_ibge_code_06'].map(int)
city_state = city_state.set_index('city_ibge_code_06')['state'].to_dict()
code7_6['city_ibge_code_07'] = code7_6['city_ibge_code_07'].map(int)
code7_6['city_ibge_code_06'] = code7_6['city_ibge_code_06'].map(int)
code7_6 = code7_6.set_index('city_ibge_code_07').to_dict()['city_ibge_code_06']

{1100015: 110001,
 1100023: 110002,
 1100031: 110003,
 1100049: 110004,
 1100056: 110005,
 1100064: 110006,
 1100072: 110007,
 1100080: 110008,
 1100098: 110009,
 1100106: 110010,
 1100114: 110011,
 1100122: 110012,
 1100130: 110013,
 1100148: 110014,
 1100155: 110015,
 1100189: 110018,
 1100205: 110020,
 1100254: 110025,
 1100262: 110026,
 1100288: 110028,
 1100296: 110029,
 1100304: 110030,
 1100320: 110032,
 1100338: 110033,
 1100346: 110034,
 1100379: 110037,
 1100403: 110040,
 1100452: 110045,
 1100502: 110050,
 1100601: 110060,
 1100700: 110070,
 1100809: 110080,
 1100908: 110090,
 1100924: 110092,
 1100940: 110094,
 1101005: 110100,
 1101104: 110110,
 1101203: 110120,
 1101302: 110130,
 1101401: 110140,
 1101435: 110143,
 1101450: 110145,
 1101468: 110146,
 1101476: 110147,
 1101484: 110148,
 1101492: 110149,
 1101500: 110150,
 1101559: 110155,
 1101609: 110160,
 1101708: 110170,
 1101757: 110175,
 1101807: 110180,
 1200013: 120001,
 1200054: 120005,
 1200104: 120010,
 1200138: 

### BRASIL IO

In [7]:
brasil_io.dropna(subset=['city_ibge_code_07'],inplace=True)
brasil_io['city_ibge_code_07'] = brasil_io['city_ibge_code_07'].map(int).map(code7_6)
brasil_io['date'] = pd.to_datetime(brasil_io['date'])
brasil_io.rename({'city_ibge_code_07':'city_ibge_code_06'},axis=1,inplace=True)
brasil_io['mmyyyy'] = brasil_io['date'].dt.month.map(str) + '/' + brasil_io['date'].dt.year.map(str)

### STRINGENCY INDEX

In [8]:
str_index['date'] = pd.to_datetime(str_index['date'])
str_index['mmyyyy'] = str_index['date'].dt.month.map(str) + '/' + str_index['date'].dt.year.map(str)
str_index = str_index.groupby(['state','mmyyyy']).mean().reset_index()

### VACCINE

In [9]:
vacc = vacc[vacc['city_ibge_code_06']!='']
vacc['date'] = pd.to_datetime(vacc['date'])
vacc['city_ibge_code_06'] = vacc['city_ibge_code_06'].map(int)
vacc['mmyyyy'] = vacc['date'].dt.month.map(str) + '/' + vacc['date'].dt.year.map(str)
vacc = vacc.groupby(['city_ibge_code_06','mmyyyy']).sum().reset_index()
vacc = vacc.groupby(['city_ibge_code_06','mmyyyy']).sum().reset_index()

,city_ibge_code_06,mmyyyy,doses
0,110001,1/2022,474.0
1,110001,10/2021,2149.0
2,110001,11/2021,4593.0
3,110001,12/2021,1000.0
4,110001,2/2021,204.0
...,...,...,...
89358,530010,5/2022,122672.0
89359,530010,6/2021,34640.0
89360,530010,7/2021,233031.0
89361,530010,8/2021,259861.0


### MERGE TABLES

In [11]:
dataset = brasil_io.merge(right=vacc,on=['city_ibge_code_06','mmyyyy'])
dataset['state'] = dataset['city_ibge_code_06'].map(city_state)
dataset.merge(right=str_index,on=['mmyyyy','state'])

,state,city_name,city_ibge_code_06,date,deaths,estimated_population,mmyyyy,doses,stringency_index
0,GO,Abadia de Goiás,520005,2021-02-01,10,8958.0,2/2021,93.0,46.300000
1,GO,Abadia de Goiás,520005,2021-02-02,10,8958.0,2/2021,93.0,46.300000
2,GO,Abadia de Goiás,520005,2021-02-03,10,8958.0,2/2021,93.0,46.300000
3,GO,Abadia de Goiás,520005,2021-02-04,11,8958.0,2/2021,93.0,46.300000
4,GO,Abadia de Goiás,520005,2021-02-05,11,8958.0,2/2021,93.0,46.300000
...,...,...,...,...,...,...,...,...,...
2355175,RO,Vilhena,110030,2021-01-27,107,102211.0,1/2021,1.0,52.687742
2355176,RO,Vilhena,110030,2021-01-28,111,102211.0,1/2021,1.0,52.687742
2355177,RO,Vilhena,110030,2021-01-29,111,102211.0,1/2021,1.0,52.687742
2355178,RO,Vilhena,110030,2021-01-30,111,102211.0,1/2021,1.0,52.687742


### EXPORT DATA

In [ ]:
dataset.to_csv("dataset.csv")